In [90]:
# !pip install Shapely
import pyproj
from pyproj import Proj, transform
import geopandas as gpd
from pyproj import Transformer
from shapely.geometry import MultiLineString, MultiPolygon, Polygon, LineString, Point
import matplotlib.pyplot as plt
from shapely.ops import transform

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
# plt.rc("font", family="NanumGothic") # 라이브러리 불러오기와 함께 한번만 실행

# plt.plot([1, 2, 3])
# plt.title("한글")
# plt.show()

In [91]:
df = gpd.read_file("./dotsdots.shp")
df = df[['RN','geometry']]
print(df.shape)
df.head()

(471, 2)


,RN,geometry
0,장군봉7길,POINT (194749.831 442263.232)
1,청룡7길,POINT (195272.057 442265.700)
2,남부순환로214길,POINT (195357.725 442327.845)
3,장군봉1길,POINT (194902.676 442470.017)
4,청룡12길,POINT (195173.622 442037.840)


### 참고. 좌표계 다른 경우

In [92]:
crs_from = pyproj.CRS("EPSG:5181")
crs_to = pyproj.CRS("EPSG:4326")  # WGS 84 (위도, 경도)

# 좌표 변환 함수 정의
def transform_coordinates(geometry):
    transformer = pyproj.Transformer.from_crs(crs_from, crs_to, always_xy=True)
    lon, lat = transformer.transform(geometry.x, geometry.y)
    return Point(lon, lat)

# Geometry 컬럼 변환
df['geometry'] = df['geometry'].apply(transform_coordinates)
print(df.crs)

df.head()

EPSG:5181


,RN,geometry
0,장군봉7길,POINT (126.941 37.480)
1,청룡7길,POINT (126.947 37.480)
2,남부순환로214길,POINT (126.948 37.480)
3,장군봉1길,POINT (126.942 37.482)
4,청룡12길,POINT (126.945 37.478)


In [93]:
# df.to_csv('selected_dong_dots.csv', encoding = 'utf-8-sig', index = False)

### 0. geo데이터프레임으로 변경

In [95]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   RN        471 non-null    object  
 1   geometry  471 non-null    geometry
dtypes: geometry(1), object(1)
memory usage: 7.5+ KB


### 1. 기본 카테고리별로 개수세기
- queries = ['대형마트', '편의점', '어린이집', '학교', '학원', '주차장', '주유소', '지하철역', '은행', '문화시설', '중개업소', '공공기관', '관광명소', '숙박', '음식점', '카페', '병원', '약국']

In [100]:
# API 요청을 위한 함수 정의
def find_nearby_places(row, category_code):
    latitude = row.geometry.y
    longitude = row.geometry.x
    
    radius = 200  # 원하는 반경 설정
    url = f"https://dapi.kakao.com/v2/local/search/category.json?category_group_code={category_code}&x={longitude}&y={latitude}&radius={radius}"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    
    response = requests.get(url, headers=headers)
    data = response.json()
    
    place_count = len(data.get("documents", []))
    return place_count

# 데이터프레임 및 카테고리 그룹 코드 정의
category_group_codes = ['MT1', 'CS2', 'PS3', 'SC4', 'AC5', 'PK6', 'OL7', 'SW8', 'BK9', 'CT1', 'AG2', 'PO3', 'AT4', 'AD5', 'FD6', 'CE7', 'HP8', 'PM9']

# 각 카테고리별로 개수를 세고 결과 데이터프레임 생성
result_df = gdf.copy()
for category_code in category_group_codes:
    place_count_list = []
    for index, row in result_df.iterrows():
        place_count = find_nearby_places(row, category_code)
        place_count_list.append(place_count)
    result_df[f"{category_code}_개수"] = place_count_list

result_df.head()

,RN,geometry,MT1_개수,CS2_개수,PS3_개수,SC4_개수,AC5_개수,PK6_개수,OL7_개수,SW8_개수,BK9_개수,CT1_개수,AG2_개수,PO3_개수,AT4_개수,AD5_개수,FD6_개수,CE7_개수,HP8_개수,PM9_개수
0,장군봉7길,POINT (126.941 37.480),0,6,3,0,2,0,0,0,4,0,15,2,0,0,15,9,2,1
1,청룡7길,POINT (126.947 37.480),0,8,2,0,9,1,0,0,6,0,15,0,0,4,15,15,4,2
2,남부순환로214길,POINT (126.948 37.480),1,8,1,0,12,2,0,0,12,0,15,0,0,6,15,15,7,3
3,장군봉1길,POINT (126.942 37.482),0,6,2,1,15,4,0,1,9,0,15,0,1,0,15,15,15,9
4,청룡12길,POINT (126.945 37.478),0,4,1,2,3,1,0,0,1,0,15,0,0,0,15,7,3,1


In [ ]:
result_df.columns = ['RN', 'geometry', '대형마트_개수', '편의점_개수', '어린이집_개수', '학교_개수', '학원_개수',
       '주차장_개수', '주유소_개수', '지하철역_개수', '은행_개수', '문화시설_개수', '중개업소_개수', '공공기관_개수',
       '관광명소_개수', '숙박_개수', '음식점_개수', '카페_개수', '병원_개수', '약국_개수']

In [ ]:
result_df.head(1)
# 126.942 37.482 카페 개수로 검증 완료

### 2. 기타 시설 추가
- 헬스클럽과 코인워시 예시

In [ ]:
# API 요청 함수 정의
query = ['헬스클럽', '셀프빨래방']
def find_nearby_spots(row, query):
    latitude = row.geometry.y
    longitude = row.geometry.x
    radius = 200
    
    url = f"https://dapi.kakao.com/v2/local/search/keyword.json?query={query}&x={longitude}&y={latitude}&radius={radius}"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    
    response = requests.get(url, headers=headers)
    data = response.json()
    
    place_count = len(data.get("documents", []))
    return place_count


for query_item in query:
    place_count_list = []
    for index, row in result_df.iterrows():
        place_count = find_nearby_spots(row, query_item)
        place_count_list.append(place_count)
    result_df[f"{query_item}_개수"] = place_count_list

result_df.head()